In [1]:
!pip install --upgrade --no-cache-dir gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [2]:
import gdown
!gdown --fuzzy 'https://drive.google.com/file/d/1q4GkMwcuQ8kDGvnW3ncGTxWhiJ0q7yDR/view?usp=sharing'

Downloading...
From: https://drive.google.com/uc?id=1q4GkMwcuQ8kDGvnW3ncGTxWhiJ0q7yDR
To: /content/train (1).csv
100% 68.8M/68.8M [00:00<00:00, 96.9MB/s]


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
df = pd.read_csv("/content/train (1).csv")
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
df.shape

(159571, 8)

In [6]:
df.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [8]:
from tensorflow.keras.layers import TextVectorization

In [11]:
x = df['comment_text']
y = df[df.columns[2:]].values

In [12]:
max_features = 200000

In [14]:
vectorizer = TextVectorization(max_tokens = max_features ,
                               output_sequence_length = 180 ,
                               output_mode = 'int')

In [15]:
vectorizer.adapt(x.values)

In [16]:
vectorizer_text = vectorizer(x.values)

In [17]:
dataset = tf.data.Dataset.from_tensor_slices((vectorizer_text , y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(12)
dataset = dataset.prefetch(8)

In [18]:
tra = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
tes = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Bidirectional , Embedding , Dense

In [20]:
model = Sequential()

model.add(Embedding(max_features , 32))

model.add(Bidirectional(LSTM(32 , activation = 'tanh')))

model.add(Dense(128 , activation = 'relu'))
model.add(Dense(256 , activation = 'relu'))
model.add(Dense(512 , activation = 'relu'))

model.add(Dense(6 , activation = 'sigmoid'))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400000   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dense_3 (Dense)             (None, 6)                 3078      
                                                        

In [24]:
model.compile(optimizer = 'Adam' , loss = 'BinaryCrossentropy' , metrics = ['Accuracy'])

In [26]:
hist = model.fit(tra , validation_data = val , epochs = 1)

9308/9308 [==============================] - 1997s 215ms/step - loss: 0.0507 - Accuracy: 0.9925 - val_loss: 0.0441 - val_Accuracy: 0.9938


In [42]:
input_text = vectorizer(["I like you"])

In [43]:
res = model.predict(input_text)

1/1 [==============================] - 0s 34ms/step


In [44]:
(res>0.5).astype('int')

array([[0, 0, 0, 0, 0, 0]])